# import packages and libraries

In [31]:
#Import Packages
import time
import pandas as pd
import os
import gspread

In [ ]:
#Import Packages
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from google.auth import exceptions
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe

# URL 

In [ ]:
# Linkedin URL 

url='link to your linkedin link'

# automated and open URL 

In [ ]:
chrome_driver_path = r'' # chrome driver path

# Chrome service with the specified executable path
chrome_service = Service(chrome_driver_path)

# Initialize the Chrome driver using the Chrome service
driver = webdriver.Chrome(service=chrome_service)
driver.implicitly_wait(10)
driver.get(url)

# Job count 

In [ ]:
#Find number of job listings
job_count_element = driver.find_element(By.CLASS_NAME, 'results-context-header__job-count') # count the job list
job_count_text = job_count_element.text
print("Number of job listings:", job_count_text)

# Type casting 

In [ ]:
type(job_count_text)

In [ ]:
# Remove non-numeric characters
cleaned_text = ''.join(filter(str.isdigit, job_count_text))

# Convert to numeric
n = pd.to_numeric(cleaned_text)

print("Number of job listings (numeric):", n)

In [ ]:
n

# Implemented automated infinite scrolling and a custom button-based job list filter

In [ ]:
# custom limit for the number of clicks

custom_click_limit = 30

i = 2
while i <= custom_click_limit:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # for scrolling
    time.sleep(2)

    try:
        btn = driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']") # click button
        driver.execute_script("arguments[0].click();", btn)
        print(f"Clicked 'See more jobs' button {i - 1} times.")
        time.sleep(2)  # short pause after clicking the button

    except NoSuchElementException:
        print("No more 'See more jobs' button found. Exiting loop.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        time.sleep(3)

    i += 1

print("Number of times 'See more jobs' button was clicked:", i - 2 if i > 2 else 0)

# give time for the new jobs to load
time.sleep(4)


# Lists to store job details

In [ ]:
# Lists to store job details

company_names = []
job_titles = []
job_locations = []
job_post_urls = []
Job_Salary = []
Job_Info = []
Job_ListTime = []

# Scraping data from Linkedin

# scraping data with minimum lenght

In [ ]:
# The code will iteratively scrape job listings and populate a storage list until it encounters a terminating condition indicated by None values

try:
    for i in range(n):
        # Use find_elements instead of find_element to get a list of elements
        company_name_elements = driver.find_elements(By.CLASS_NAME, 'base-search-card__subtitle')
        job_title_elements = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')
        job_location_elements = driver.find_elements(By.CLASS_NAME, 'job-search-card__location')
        job_post_url_elements = driver.find_elements(By.CSS_SELECTOR, 'a.base-card__full-link')
        job_salary_elements = driver.find_elements(By.CLASS_NAME, 'job-search-card__salary-info')
        job_info_elements = driver.find_elements(By.CLASS_NAME, 'result-benefits__text')
        list_time_elements = driver.find_elements(By.CLASS_NAME, 'job-search-card__listdate')

        # Extract data for 'Company Name'
        company_name_data = [element.text if i < len(company_name_elements) else "None" for i, element in enumerate(company_name_elements)]
        company_names.extend(company_name_data)

        # Extract data for 'Job Title'
        job_title_data = [element.text if i < len(job_title_elements) else "None" for i, element in enumerate(job_title_elements)]
        job_titles.extend(job_title_data)

        # Extract data for 'Job Location'
        job_location_data = [element.text if i < len(job_location_elements) else "None" for i, element in enumerate(job_location_elements)]
        job_locations.extend(job_location_data)

        # Extract data for 'Job Post URL'
        job_post_url_data = [element.get_attribute('href') if i < len(job_post_url_elements) else "None" for i, element in enumerate(job_post_url_elements)]
        job_post_urls.extend(job_post_url_data)

        # Extract data for 'Job Salary'
        job_salary_data = [element.text if i < len(job_salary_elements) else "None" for i, element in enumerate(job_salary_elements)]
        Job_Salary.extend(job_salary_data)

        # Extract data for 'Job Info'
        job_info_data = [element.text if i < len(job_info_elements) else "None" for i, element in enumerate(job_info_elements)]
        Job_Info.extend(job_info_data)

        # Extract data for 'List Time'
        list_time_data = [element.get_attribute('datetime') if i < len(list_time_elements) else "None" for i, element in enumerate(list_time_elements)]
        Job_ListTime.extend(list_time_data)
    
except IndexError:
    print("Error: Index out of range. There may be an issue with the number of jobs loaded.")

except Exception as e:
    print(f"An error occurred while extracting job details: {e}")

finally:
    # Check if all lists have the same length
    min_length = min(len(company_names), len(job_titles), len(job_locations), len(job_post_urls), len(Job_Salary), len(Job_Info), len(Job_ListTime))

    # Trim lists to the minimum length
    company_names = company_names[:min_length]
    job_titles = job_titles[:min_length]
    job_locations = job_locations[:min_length]
    job_post_urls = job_post_urls[:min_length]
    Job_Salary = Job_Salary[:min_length]
    Job_Info = Job_Info[:min_length]
    Job_ListTime = Job_ListTime[:min_length]

    # Proceed to create DataFrame
    data = {
        'Company Name': company_names,
        'Job Title': job_titles,
        'Location': job_locations,
        'Job Post URL': job_post_urls,
        'Salary': Job_Salary,
        'Job Info': Job_Info,
        'List Time': Job_ListTime
    }

    df = pd.DataFrame(data)
    print(df)


# scraping job list with maximum lenght

In [ ]:
# The code will iteratively scrape job listings and populate a storage list until it encounters a terminating condition indicated by None values

try:
    for i in range(n):
        # Use find_elements instead of find_element to get a list of elements
        company_name_elements = driver.find_elements(By.CLASS_NAME, 'base-search-card__subtitle')
        job_title_elements = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')
        job_location_elements = driver.find_elements(By.CLASS_NAME, 'job-search-card__location')
        job_post_url_elements = driver.find_elements(By.CSS_SELECTOR, 'a.base-card__full-link')
        job_salary_elements = driver.find_elements(By.CLASS_NAME, 'job-search-card__salary-info')
        job_info_elements = driver.find_elements(By.CLASS_NAME, 'result-benefits__text')
        list_time_elements = driver.find_elements(By.CLASS_NAME, 'job-search-card__listdate')

        # Extract data for 'Company Name'
        company_name_data = [element.text if i < len(company_name_elements) else "None" for i, element in enumerate(company_name_elements)]
        company_names.extend(company_name_data)

        # Extract data for 'Job Title'
        job_title_data = [element.text if i < len(job_title_elements) else "None" for i, element in enumerate(job_title_elements)]
        job_titles.extend(job_title_data)

        # Extract data for 'Job Location'
        job_location_data = [element.text if i < len(job_location_elements) else "None" for i, element in enumerate(job_location_elements)]
        job_locations.extend(job_location_data)

        # Extract data for 'Job Post URL'
        job_post_url_data = [element.get_attribute('href') if i < len(job_post_url_elements) else "None" for i, element in enumerate(job_post_url_elements)]
        job_post_urls.extend(job_post_url_data)

        # Extract data for 'Job Salary'
        job_salary_data = [element.text if i < len(job_salary_elements) else "None" for i, element in enumerate(job_salary_elements)]
        Job_Salary.extend(job_salary_data)

        # Extract data for 'Job Info'
        job_info_data = [element.text if i < len(job_info_elements) else "None" for i, element in enumerate(job_info_elements)]
        Job_Info.extend(job_info_data)

        # Extract data for 'List Time'
        list_time_data = [element.get_attribute('datetime') if i < len(list_time_elements) else "None" for i, element in enumerate(list_time_elements)]
        Job_ListTime.extend(list_time_data)
    
except IndexError:
    print("Error: Index out of range. There may be an issue with the number of jobs loaded.")

except Exception as e:
    print(f"An error occurred while extracting job details: {e}")

finally:
    # Trim all lists to the length of the longest list
    max_length = max(len(company_names), len(job_titles), len(job_locations), len(job_post_urls), len(Job_Salary), len(Job_Info), len(Job_ListTime))

    company_names = company_names + ["None"] * (max_length - len(company_names))
    job_titles = job_titles + ["None"] * (max_length - len(job_titles))
    job_locations = job_locations + ["None"] * (max_length - len(job_locations))
    job_post_urls = job_post_urls + ["None"] * (max_length - len(job_post_urls))
    Job_Salary = Job_Salary + ["None"] * (max_length - len(Job_Salary))
    Job_Info = Job_Info + ["None"] * (max_length - len(Job_Info))
    Job_ListTime = Job_ListTime + ["None"] * (max_length - len(Job_ListTime))

    # Proceed to create DataFrame
    data = {
        'Company Name': company_names,
        'Job Title': job_titles,
        'Location': job_locations,
        'Job Post URL': job_post_urls,
        'Salary': Job_Salary,
        'Job Info': Job_Info,
        'List Time': Job_ListTime
    }

    df = pd.DataFrame(data)
    print(df)


In [ ]:
df

# Utilizing the Google Sheets API to programmatically transfer a DataFrame's contents to a Google Sheet

### go to gooogle cloud and then create new project. create service account and in tab theres key tab and then create json key and copy that json path and get the client email from that json file and connect it with spreadsheet and enable google sheet & drive API and you're ready to run below code that will connect dataframe to spreadsheet directly with spreadsheet link

In [ ]:
#Google API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"] 
# Credentials file in JSON format for accessing Google Sheets APIs
creds = Credentials.from_service_account_file(r'path of JSON key', scopes=scope)
client = gspread.authorize(creds)

# Google Spreadsheet Link 
spreadsheet = client.open_by_url('link to your spreadsheet')

#spreadsheet = client.open("name of your spreadsheet") # open by spreadsheet name
worksheet = spreadsheet.get_worksheet(0)  # first worksheet in the spreadsheet

# Clear existing data in the sheet
worksheet.clear()

# Write the DataFrame to the Google Sheet
set_with_dataframe(worksheet, df)

In [ ]:
 # Close the WebDriver

driver.quit()

# thanks (❁´◡`❁)